In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [3]:
url = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat"
names = ["user_id", "movie_id", "rating", "timestamp"]
data = pd.read_csv(url, sep="::", names=names, engine="python")

In [4]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [5]:
class RecommenderNet(nn.Module):
    def __init__(self):
        super(RecommenderNet, self).__init__()
        self.fc1 = nn.Linear(1, 16)
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        # Definindo que a função de ativação da camada convolucional é a relu.
        # Ela vai ser aplicada logo após o calculo linear da camada ser efetuado.
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = RecommenderNet()

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

In [7]:
# Para treinar a rede, é necessário criar um looping normal kkkkk
# O otimizador e a loss function são executadas nesse etapa

for epoch in range(10):
    for i in range(len(train_data)):
        user_id = train_data.iloc[i]["user_id"]
        rating = train_data.iloc[i]["rating"]
        
        # .view(-1, 1) serve para redimensionar um tensor multidimensional em unidimensional.
        # Por isso nas layers full conected, é passado como tamanho do kernel a multiplicação dos eixos do tensor
        x = torch.tensor(user_id, dtype=torch.float32).view(-1, 1)
        y = torch.tensor(rating, dtype=torch.float32).view(-1, 1)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        
        # Calcula o vetor gradiente dos tensores e armazena no vetor.
        # Quando o tensor tem a propriedade 'requires_grad' falsa, ele não calcula para tal.
        loss.backward()
        optimizer.step()

KeyboardInterrupt: 

In [ ]:
test_predictions = []
with torch.no_grad():
    for i in range(len(test_data)):
        user_id = test_data.iloc[i]["user_id"]
        x = torch.tensor(user_id, dtype=torch.float32).view(-1, 1)
        y_pred = model(x)
        test_predictions.append(y_pred.item())